## **4. Create sample event using Databricks notebook and sent to Event Hub** ##

In [ ]:
from azure.eventhub import  EventHubProducerClient, EventData
import json

# Event Hub Configuration
EVENT_HUB_CONNECTION_STRING = "connection-string-key-from-keyVault"
EVENT_HUB_NAME = "eventhub-name"

# Initialize the Event Hub producer
producer = EventHubProducerClient.from_connection_string(conn_str=EVENT_HUB_CONNECTION_STRING, eventhub_name=EVENT_HUB_NAME)

# Function to send events to Event Hub
def send_event(event):
    event_data_batch =producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

# Sample JSON event
event = {
    "event_id": 111,
    "event_name": "Test Event",
}  

# Send the Event
send_event(event)

# Close the producer
producer.close()

## **5. Configure Key vault Secrets in Databricks** ##

### **5.1. Create Key Vault SECRET SCOPE** ###
- To establish a connection between **Databricks target workspace** and **Azure Key Vault**, we need to create **Secret scope** in the Databricks workspace based on the official Azure Databricks documentation ([Secret management](https://learn.microsoft.com/en-us/azure/databricks/security/secrets/))
- A **secret scope** is collection of secrets identified by a name. Databricks recommends aligning secret scopes to roles or applications rather than individuals. There are two types of secret scope:
    - **Azure Key Vault-backed:** You can reference secrets stored in an Azure Key Vault using Azure Key Vault-backed secret scopes. Azure Key Vault-backed secret scope is a read-only interface to the Key Vault. You must manage secrets in Azure Key Vault-backed secret scopes in Azure.
    - **Databricks-backed:** A Databricks-backed secret scope is stored in an encrypted database owned and managed by Azure Databricks.

### **5.2. Update connection string to directly get the key from Azure Key Vault** ###
- When you run the notebook, it fails with denied access error message. To solve the issue, assign **"Key vault Secrets user"** role to **AzureDatabricks** service principal.
- An **Azure Databricks application service principal** is an **Azure Entra ID (Azure AD)** identity used by applications, pipelines, or automation to authenticate to Azure Databricks without a human user. It enables secure, non-interactive access for CI/CD, jobs, and integrations (e.g., ADF, Fabric, GitHub Actions). Permissions are granted through **Azure RBAC, Databricks workspace roles**, and **Unity Catalog access**, following least-privilege principles.

In [ ]:
# Getting secret value from Key Vault
eventhub_connection_string = dbutils.secrets.get(scope = "key-vault-scope-name", key = "key-vault-secret-name")
EVENT_HUB_NAME = "eventhub-name"

# Initialize the Event Hub producer
producer = EventHubProducerClient.from_connection_string(conn_str=eventhub_connection_string, eventhub_name=EVENT_HUB_NAME)

# Function to send events to Event Hub
def send_event(event):
    event_data_batch =producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

# Sample JSON event
event = {
    "event_id": 333,
    "event_name": "Key Vault Test",
}  

# Send the Event
send_event(event)

# Close the producer
producer.close()